In [1]:
from scipy.spatial import distance
from imutils import face_utils
import numpy as np
import time
import dlib
import cv2


In [2]:

face_cascade = cv2.CascadeClassifier("C:\\Users\\Lenovo\\Desktop\\face\\haarcascade_frontalface_default.xml")


In [3]:

EYE_DROWSINESS_THRESHOLD = 0.3
COUNTER = 0
EYE_ASPECT_RATIO_CONSEC_FRAMES = 50

In [4]:

def get_aspect_ratio(eye):
    vertical_1 = distance.euclidean(eye[1], eye[5])
    vertical_2 = distance.euclidean(eye[2], eye[4])
    horizontal = distance.euclidean(eye[0], eye[3])

    return (vertical_1+vertical_2)/(horizontal*2)

In [5]:

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('C:\\Users\\Lenovo\\Desktop\\face\\shape_predictor_68_face_landmarks.dat')


In [6]:

(ls, le) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rs, re) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']


In [7]:

detect = cv2.VideoCapture(0)

In [8]:
while(True):
    _ , frame = detect.read()
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    
    rects = detector(gray, 0)

    
    face_rectangle = face_cascade.detectMultiScale(gray, 1.3, 5)

    
    for (x,y,w,h) in face_rectangle:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

    for face in rects:

        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)

        
        leftEye = shape[ls:le]
        rightEye = shape[rs:re]

        #Calculate aspect ratio of both eyes
        leftEyeAspectRatio = get_aspect_ratio(leftEye)
        rightEyeAspectRatio =get_aspect_ratio(rightEye)

        eyeAspectRatio = (leftEyeAspectRatio + rightEyeAspectRatio) / 2

        
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

       
        if(eyeAspectRatio < EYE_DROWSINESS_THRESHOLD ):
            COUNTER += 1
           
   
    cv2.imshow('Video', frame)
    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break


In [9]:
detect.release()
cv2.destroyAllWindows()
